# Homework 3 - Recurrent Neural Networks

In this part of the homework we are going to work with Recurrent Neural Networks, in particular GRU. One of the greatest things that Recurrent Neural Networks can do when working with sequences is retaining data from several timesteps in the past. We are going to explore that property by constructing an 'echo' Recurrent Neural Network.

The goal here is to make a model that given a sequence of letters or digits will output that same sequence, but with a certain delay. Let's say the input is a string 'abacaba', we want the model to not output anything for 3 steps (delay length), and then output the original string step by step, except the last 3 characters. So, target output is then 'XXXabac', where 'X' is empty output.

This is similar to [this notebook](https://github.com/Atcold/pytorch-Deep-Learning/blob/master/09-echo_data.ipynb) (which you should refer to when doing this assignment), except we're working not with a binary string, but with a sequence of integers between 0 and some N. In our case N is 26, which is the number of letters in the alphabet.

## Dataset

Let's implement the dataset. In our case, the data is basically infinite, as we can always generate more examples on the fly, so don't need to load anything from disk.

In [13]:
import random
import string

import torch

# Max value of the generated integer. 26 is chosen becuase it's
# the number of letters in English alphabet.
N = 26


def idx_to_onehot(x, k=N+1):
  """ Converts the generated integers to one-hot vectors """
  ones = torch.sparse.torch.eye(k).to(device)
  shape = x.shape
  x = x.to(device)
  res = ones.index_select(0, x.view(-1).type(torch.int64))
  return res.view(*shape, res.shape[-1])


class EchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, delay=4, seq_length=15, size=1000):
    self.delay = delay
    self.seq_length = seq_length
    self.size = size
  
  def __len__(self):
    return self.size

  def __iter__(self):
    """ Iterable dataset doesn't have to implement __getitem__.
        Instead, we only need to implement __iter__ to return
        an iterator (or generator).
    """
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      result = torch.cat((torch.zeros(self.delay), seq[:self.seq_length - self.delay])).type(torch.int64)
      yield seq, result

DELAY = 4
#DATASET_SIZE = 200000
DATASET_SIZE = 10000
ds = EchoDataset(delay=DELAY, size=DATASET_SIZE)

## Model

Now, we want to implement the model. For our purposes, we want to use GRU. The architecture consists of GRU and a decoder. Decoder is responsible for decoding the GRU hidden state to yield a predicting for the next output. The parts you are responsible for filling with your code are marked with `TODO`. 

In [14]:
import torch.nn as nn

class GRUMemory(torch.nn.Module):

  def __init__(self, hidden_size):
    super(GRUMemory, self).__init__()
    #TODO: initialize your submodules
    self.hidden_dim = hidden_size
    self.n_layers = 1

    self.gru = nn.GRU(
        input_size=27, 
        hidden_size=hidden_size,
        num_layers=self.n_layers,
        batch_first=True
    )

    self.decoder = nn.Sequential(
      nn.Linear(hidden_size, out_features=27),
      #nn.Softmax(dim=2)    
    )
    
  def init_hidden(self, batch_size):
    weight = next(self.parameters()).data
    hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
    return hidden

  def forward(self, x, hidden):
    # inputs: x - input tensor of shape (batch_size, seq_length, N+1)
    # returns:
    # logits (scores for softmax) of shape (batch size, seq_length, N + 1)
    # TODO implement forward pass
    x, hidden = self.gru(x, hidden)
    x = self.decoder(x)
    return x, hidden

  @torch.no_grad()
  def test_run(self, s):
    # This function accepts one string s containing lowercase characters a-z. 
    # You need to map those characters to one-hot encodings, 
    # then get the result from your network, and then convert the output 
    # back to a string of the same length, with 0 mapped to ' ', 
    # and 1-26 mapped to a-z.
    letters = [' '] + [chr(ord('a')+x) for x in range(26)]
    encoded = torch.Tensor([ord(c)-96 if c!=' ' else 0 for c in s])
    #print(encoded)
    one_hot = torch.unsqueeze(idx_to_onehot(encoded), 0).to(device)
    h = model.init_hidden(1).to(device)
    
    output, h = model(one_hot, h)
    #print(output)
    output = output.view(-1, 27)
    output_num = torch.argmax(output, dim=1)
    #print(output_num)
    output_str = ''.join([letters[x] for x in output_num])
    #print(f'input: {s}\noutput: {output_str}')
    return output_str    


## Training
Below you need to implement the training of the model. We give you more freedom as for the implementation. The two limitations are that it has to execute within 10 minutes, and that error rate should be below 1%.

In [15]:
def test_model(model, sequence_length=15):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  D = DELAY
  total = 0
  correct = 0
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(random.randint(15, 25))])
    result = model.test_run(s)
    for c1, c2 in zip(s[:-D], result[D:]):
      correct += int(c1 == c2)
    total += len(s) - D

  return correct / total

In [16]:
import time
import torch.optim
from tqdm.notebook import tqdm
start_time = time.time()

N_EPOCHS = 15

# TODO
model = GRUMemory(hidden_size=128)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)
correct = 0
h = None

if torch.cuda.is_available():
  model = model.cuda()
  criterion = criterion.cuda()
  device = torch.device("cuda:0")
else:
  device = torch.device("cpu")

model.train()
for i in range(N_EPOCHS):
  loss = 0
  for seq, result in tqdm(ds):
    seq, result = torch.unsqueeze(idx_to_onehot(seq), 0).to(device), result.to(device)
    
    optimizer.zero_grad()
    if h is not None: h.detach_()

    outputs, h = model(seq, h)
    loss = criterion(torch.squeeze(outputs).view(-1, 27), result)
    loss.backward()
    optimizer.step()
  print(f'Epoch: {i} loss: {loss.item()}')

end_time = time.time()
duration = end_time - start_time
accuracy = test_model(model)
assert duration < 600, 'execution took f{duration:.2f} seconds, which longer than 10 mins'
assert accuracy > 0.99, f'accuracy is too low, got {accuracy}, need 0.99'
print('tests passed')

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 0 loss: 0.8160871863365173


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 1 loss: 0.8145371079444885


  0%|          | 0/10000 [00:00<?, ?it/s]

...

Epoch: 12 loss: 0.5906121730804443


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 13 loss: 0.5180311799049377


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch: 14 loss: 0.5887945890426636
tests passed


## Variable delay model

Now, to make this more complicated, we want to have varialbe delay. So, now, the goal is to transform a sequence of pairs (character, delay) into a character sequence with given delay. Delay stays constant within one sequence.

### Dataset
As before, we first implement the dataset:

In [2]:
import torch
import torch.nn as nn
import random
N=26

In [3]:
class VariableDelayEchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, max_delay=8, seq_length=20, size=1000):
    self.max_delay = max_delay
    self.seq_length = seq_length
    self.size = size
  
  def __len__(self):
    return self.size

  def __iter__(self):
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      delay = random.randint(0, self.max_delay)
      result = torch.cat((torch.zeros(delay), seq[:self.seq_length - delay])).type(torch.int64)
      yield seq, delay, result

### Model

And the model.

In [9]:
class VariableDelayGRUMemory(torch.nn.Module):

  def __init__(self, hidden_size, max_delay):
    super(VariableDelayGRUMemory, self).__init__()
    self.hidden_dim = hidden_size
    self.max_delay = max_delay
    self.n_layers = 1
    self.gru = nn.GRU(
        input_size=N+1+self.max_delay+1, 
        hidden_size=hidden_size,
        num_layers=self.n_layers,
        batch_first=True,
        #bidirectional=True
    )

    self.decoder = nn.Sequential(
      nn.Linear(hidden_size+self.max_delay+1, 256),
      #nn.ReLU(inplace=True),
      # nn.Tanh(),
      # nn.Linear(128, 64),
      # #nn.ReLU(inplace=True),
      nn.ReLU(inplace=True),
      nn.Linear(256, out_features=27),      
      #nn.Softmax(dim=2)    
    )

    self.conv = nn.Sequential(
        nn.Conv1d(hidden_size+self.max_delay+1, hidden_size+self.max_delay+1, kernel_size=self.max_delay+1, padding='same'),
        nn.ReLU(inplace=True)
    )
    # self.decoder3 = nn.Sequential(
    #     nn.Linear(hidden_size, 128),
    #     nn.ReLU(inplace=True)
    #     nn.ReLU(128,64)
    # )
    # self.shortcut = nn.Sequential(
    #     nn.Linear(1, self.hidden_dim),
    #     nn.ReLU(inplace=True)
    #   )
    
  def forward(self, x, hidden, delays):
    # inputs:
    # x - tensor of shape (batch size, seq length, N + 1)
    # delays - tensor of shape (batch size)
    # returns:
    # logits (scores for softmax) of shape (batch size, seq_length, N + 1)
    seq_len = x.shape[1]
    delay = idx_to_onehot(delays, self.max_delay+1).repeat(seq_len,1).view(1, seq_len, self.max_delay+1).to(device)
    x = torch.cat((delay, x),dim=-1).to(device)
    x, hidden = self.gru(x, hidden)
    #x = torch.cat((x, torch.tensor([delays]).repeat(seq_len, 1).view(1, seq_len, 1).to(device)), dim=-1)
    #delays = delays.to(device)
    #delays = idx_to_onehot(delays, k=self.max_delay+1).to(device)
    #x = x + self.shortcut(delays).to(device)
    #x = self.decoder(x)
    #print(x.shape)
    #x = x.view(1, self.hidden_dim, seq_len).to(device)
    x = torch.cat((x, delay), dim=-1).to(device)
    x = x.permute(0,2,1).to(device)
    #x = torch.cat((torch.zeros(1,128,self.max_delay), x), dim=-1)
    #print(x.shape)
    x = self.conv(x)
    #print(x.shape)
    x = x.permute(0,2,1).to(device)
    x = self.decoder(x)
    return x, hidden
  
  
  def init_hidden(self, batch_size):
    weight = next(self.parameters()).data
    hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
    return hidden
  

  @torch.no_grad()
  def test_run(self, s, delay):
    # This function accepts one string s containing lowercase characters a-z, 
    # and a delay - the desired output delay.
    # You need to map those characters to one-hot encodings, 
    # then get the result from your network, and then convert the output 
    # back to a string of the same length, with 0 mapped to ' ', 
    # and 1-26 mapped to a-z.

    letters = [' '] + [chr(ord('a')+x) for x in range(26)]
    encoded = torch.Tensor([ord(c)-96 if c!=' ' else 0 for c in s])
    #print(encoded)
    one_hot = torch.unsqueeze(idx_to_onehot(encoded), 0).to(device)
    h = model.init_hidden(1).to(device)
    delay = torch.Tensor([delay]).to(device)
    output, h = model(one_hot, h, delay)
    #print(output)
    output = output.view(-1, 27)
    output_num = torch.argmax(output, dim=1)
    #print(output_num)
    output_str = ''.join([letters[x] for x in output_num])
    #print(f'input: {s} with delay {delay.item()}\noutput: {output_str}')
    return output_str


### Train

As before, you're free to do what you want, as long as training finishes within 10 minutes and accuracy is above 0.99 for delays between 0 and 8.

In [10]:
def test_variable_delay_model(model, seq_length=20):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(seq_length)])
    d = random.randint(0, model.max_delay)
    result = model.test_run(s, d)
    if d > 0:
      z = zip(s[:-d], result[d:])
    else:
      z = zip(s, result)
    for c1, c2 in z:
      correct += int(c1 == c2)
    total += len(s) - d

  return correct / total

In [12]:
import time
from tqdm.notebook import tqdm
start_time = time.time()

MAX_DELAY = 8
SEQ_LENGTH = 20
N_EPOCHS = 15

vds = VariableDelayEchoDataset(size=5000)
# TODO: implement model training here.
model = VariableDelayGRUMemory(hidden_size=256, max_delay=MAX_DELAY)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)
# optimizer = torch.optim.RMSprop(model.parameters(), lr=0.0009)
correct = 0
h = None

if torch.cuda.is_available():
  model = model.cuda()
  criterion = criterion.cuda()
  device = torch.device("cuda:0")
else:
  device = torch.device("cpu")

model.train()
cnt = 0
for i in range(N_EPOCHS):
  loss = 0
  for seq, delay, result in tqdm(vds):
    delay = torch.Tensor([delay]).to(device)
    #h += delay.repeat(1, model.hidden_dim).view(1, -1)
    seq, delay, result = torch.unsqueeze(idx_to_onehot(seq), 0).to(device), delay, result.to(device)
    
    optimizer.zero_grad()
    if h is not None: h.detach_()

    outputs, h = model(seq, h, delay)
    loss = criterion(torch.squeeze(outputs).view(-1, 27), result)
    loss.backward()
    optimizer.step()
  # if loss < 0.5:
  #   cnt+=1
  # else:
  #   cnt=0
  # if cnt == 5:
  #   break
  print(f'Epoch: {i} loss: {loss.item()}')



end_time = time.time()
variable_accuracy = test_variable_delay_model(model)
assert variable_accuracy > 0.99, f'accuracy is too low, got {variable_accuracy}'
assert end_time - start_time < 600, 'executing took longer than 10 mins'
print('tests passed')

  0%|          | 0/5000 [00:00<?, ?it/s]

Epoch: 0 loss: 0.07110502570867538


  0%|          | 0/5000 [00:00<?, ?it/s]

Epoch: 1 loss: 0.17477209866046906


  0%|          | 0/5000 [00:00<?, ?it/s]

...

Epoch: 12 loss: 0.0


  0%|          | 0/5000 [00:00<?, ?it/s]

Epoch: 13 loss: 0.00022242276463657618


  0%|          | 0/5000 [00:00<?, ?it/s]

Epoch: 14 loss: 0.0
tests passed


In [ ]:
|#10 elochs w/ batch-size 5000 lr 0.003
#hidden 64 decoder 64-64-27 0.7642
#hidden 128 decoder 64-64-27 0.7805
#batchsize 10000 0.75

#lr0.01 0.27
#lr0.0005 0.78

#repeat delay to the size of hidden dim 128 batch_size 10000 lr 0.0005 10 epochs 0.80
#hidden_dim 64, lr 0.001 10 epochs batch_size 5000 0.59
#h 64, lr 0.0005 momentum 0.2 10 epochs with 5000 batch_size  0.60
#h 64, lr 0.0005 momentum 0.8 10 epochs with 5000 batch_size  0.66
#h 64, lr 0.0005 momentum 0.9 10 epochs with 10000 batch_size  0.38
#h 64, lr 0.0005 momentum 0.8 10 epochs with 5000 batch_size single linear layer as decoder  0.65

#h 64 enhanced delay-onehot-encoded, 128-64-27 decoder batch_size 5000 lr 0.0005 momentum 0.8 10 epochs

In [ ]:
# *rmsprop 5 epoch lr 0.0015 0.8887
# rmsprop 5 epoch lr 0.0017 0.8515
# *rmsprop 5 epoch lr 0.00158 0.90
# *rmsprop 15 epoch lr 0.00158 0.9709
# *rmsprop 15 epoch lr 0.00155 0.9642
# *rmsprop 10 epoch lr 0.00158 0.96
# *rmsprop 10*10000 lr 0.00157 0.9772
# adam lr 0.00158 0.86
# adam lr 0.0015 0.32? 0.476